## Тестирование точности работы нейронной сети

### Загружаем существующую модель

Загружаем уже готовую или обученную в прошлом модель из [./trained_models](./trained_models)

In [ ]:
from bert_classifier import BertClassifier

classifier = BertClassifier(
    model_path='cointegrated/rubert-tiny',          # Путь до репозитория с дообучаемой моделью
    tokenizer_path='cointegrated/rubert-tiny',      # Путь до токенизатора
    n_classes=41,                                   # Количество классов для обучения
    epochs=60,                                      # Количество эпох обучения
    max_len=512,                                    # Максимальный размер текста
    model_save_path='./output/model.pt'             # Путь до сохранения модели
)

classifier.load_model('./trained_models/model_v0_5.pt')

### Используя тестовый датасет, предсказываем группы направлений

* Загружаем тестовый датасет
* Предсказываем группы направлений
* Сравниваем результаты
* Сохраняем отчёт

In [ ]:
import pandas as pd
import time

### some data preparation logic
test_data  = pd.read_csv('./dataset/test.csv')

texts = list(test_data['groups'])
labels = list(test_data['code'])

predictions = [classifier.predict(t) for t in texts]

report = []     # form report variable

for t, l, p in zip(texts, labels, predictions):
    report.append({
        "real_group": l,
        "predicted_group": p['prediction'],
        "isEqual": ("no", "yes")[l == p['prediction']],
        "groups_set": t
    })
 
### display results (uncomment what u need)
report_df = pd.DataFrame(report)
report_df.to_csv(f"./reports/test{time.time()}.csv", index_label="id", sep=";", encoding="cp1251")

print(f"Точность предсказаний: {(report_df[(report_df['isEqual']=='yes')].count()['isEqual']/report_df.__len__())}")